# **l'algorithem de Lamport**
#### Mettre en place une simulation simple des horloges logiques de Lamport dans un système distribué à trois processus Chaque événement sera représenté par un objet contenant :
- le processus concerné,
- le type d’événement (local, envoi, réception),
- l’estampille logique (Lamport timestamp),
- et son historique causal (ensemble des événements qui le précèdent).


In [ ]:
#    Classe représentant un événement dans un système distribué.
class Event:
    def __init__(self, name, process, event_type, timestamp, partner=None, history=None):
        self.name = name
        self.process = process
        self.type = event_type
        self.timestamp = timestamp
        self.partner = partner
        self.history = history.copy() if history else []

    def __repr__(self):
        return self.name

#    Classe représentant un processus doté d'une horloge logique de Lamport.
class LamportClock:
    def __init__(self, name):
        self.name = name
        self.time = 0
        self.last_event = None  
        self.events = []        
    def local(self, event_name):
        self.time += 1
        history = []
        if self.last_event:
            history = self.last_event.history.copy()
            history.append(self.last_event.name)
        ev = Event(event_name, self, "local", self.time, history=history)
        self.events.append(event_name)
        self.last_event = ev
        print(f"{self.name}: Local {event_name} -> Horloge = {self.time}")
        return ev

    def emission(self, event_name, receiver):
        self.time += 1
        history = []
        if self.last_event:
            history = self.last_event.history.copy()
            history.append(self.last_event.name)
        ev = Event(event_name, self, "send", self.time, partner=receiver, history=history)
        self.events.append(event_name)
        self.last_event = ev
        print(f"{self.name}: Envoi {event_name} à {receiver.name} -> Horloge = {self.time}")
        return ev

    def reception(self, event_name, sender, received_time, sender_last_event):
        self.time = max(self.time, received_time) + 1
        history = []
        if self.last_event:
            history = self.last_event.history.copy()
            history.append(self.last_event.name)
        if sender_last_event:
            for e in sender_last_event.history + [sender_last_event.name]:
                if e not in history:
                    history.append(e)
        ev = Event(event_name, self, "receive", self.time, partner=sender, history=history)
        self.events.append(event_name)
        self.last_event = ev
        print(f"{self.name}: Réception {event_name} de {sender.name} -> Horloge = {self.time}")
        return ev


### Simulation du scénario du TP


In [ ]:
# Creation de processus
p1 = LamportClock("P1")
p2 = LamportClock("P2")
p3 = LamportClock("P3")

# creation des evenements
e1 = p1.emission("e1", p2)
e2 = p2.reception("e2", p1, p1.time, e1)
e3 = p2.emission("e3", p3)
e4 = p3.reception("e4", p2, p2.time, e3)
e5 = p1.local("e5")
e6 = p3.emission("e6", p1)
e7 = p1.reception("e7", p3, p3.time, e6)

# display de l'historique causal
events = [e1, e2, e3, e4, e5, e6, e7]
print("\n=== Historique causal de chaque événement ===")
for e in events:
    print(f"{e.name}: {e.history if e.history else 'aucun'}")


P1: Envoi e1 à P2 -> Horloge = 1
P2: Réception e2 de P1 -> Horloge = 2
P2: Envoi e3 à P3 -> Horloge = 3
P3: Réception e4 de P2 -> Horloge = 4
P1: Local e5 -> Horloge = 2
P3: Envoi e6 à P1 -> Horloge = 5
P1: Réception e7 de P3 -> Horloge = 6

=== Historique causal de chaque événement ===
e1: aucun
e2: ['e1']
e3: ['e1', 'e2']
e4: ['e1', 'e2', 'e3']
e5: ['e1']
e6: ['e1', 'e2', 'e3', 'e4']
e7: ['e1', 'e5', 'e2', 'e3', 'e4', 'e6']


# **l'algorithem de Lamport (version unifiée)**
#### Simuler les horloges logiques de Lamport dans un système distribué à plusieurs processus, en unifiant les opérations d’émission et de réception dans une seule méthode send_message().

In [ ]:
class Event:
    def __init__(self, name, process, event_type, timestamp, partner=None, history=None):
        self.name = name
        self.process = process
        self.type = event_type
        self.timestamp = timestamp
        self.partner = partner
        self.history = history.copy() if history else []

    def __repr__(self):
        return self.name


class LamportClock:
    def __init__(self, name):
        self.name = name
        self.time = 0
        self.last_event = None
        self.events = []

    def local(self, event_name):
        self.time += 1
        history = []
        if self.last_event:
            history = self.last_event.history.copy()
            history.append(self.last_event.name)
        ev = Event(event_name, self, "local", self.time, history=history)
        self.events.append(event_name)
        self.last_event = ev
        print(f"{self.name}: Local {event_name} -> Horloge = {self.time}")
        return ev

    def send_message(self, event_send_name, receiver, event_receive_name):
        """Combine emission + reception into one unified call."""
        self.time += 1
        history = []
        if self.last_event:
            history = self.last_event.history.copy()
            history.append(self.last_event.name)
        send_event = Event(event_send_name, self, "send", self.time, partner=receiver, history=history)
        self.events.append(event_send_name)
        self.last_event = send_event
        print(f"{self.name}: Envoi {event_send_name} à {receiver.name} -> Horloge = {self.time}")

        receiver_time_before = receiver.time
        receiver.time = max(receiver.time, self.time) + 1
        receiver_history = []
        if receiver.last_event:
            receiver_history = receiver.last_event.history.copy()
            receiver_history.append(receiver.last_event.name)

        merged = receiver_history.copy()
        for e in send_event.history + [send_event.name]:
            if e not in merged:
                merged.append(e)

        receive_event = Event(event_receive_name, receiver, "receive", receiver.time, partner=self, history=merged)
        receiver.events.append(event_receive_name)
        receiver.last_event = receive_event
        print(f"{receiver.name}: Réception {event_receive_name} de {self.name} -> Horloge = {receiver.time}")

        return send_event, receive_event


In [ ]:
# Creation de processus
p1 = LamportClock("P1")
p2 = LamportClock("P2")
p3 = LamportClock("P3")

# creation des evenements
e1, e2 = p1.send_message("e1", p2, "e2")   # P1→P2
e3, e4 = p2.send_message("e3", p3, "e4")   # P2→P3
e5 = p1.local("e5")                        # Local event on P1
e6, e7 = p3.send_message("e6", p1, "e7")   # P3→P1

# display de l'historique causal
events = [e1, e2, e3, e4, e5, e6, e7]
print("\n=== Historique causal de chaque événement ===")
for e in events:
    print(f"{e.name}: {e.history if e.history else 'aucun'}")


P1: Envoi e1 à P2 -> Horloge = 1
P2: Réception e2 de P1 -> Horloge = 2
P2: Envoi e3 à P3 -> Horloge = 3
P3: Réception e4 de P2 -> Horloge = 4
P1: Local e5 -> Horloge = 2
P3: Envoi e6 à P1 -> Horloge = 5
P1: Réception e7 de P3 -> Horloge = 6

=== Historique causal de chaque événement ===
e1: aucun
e2: ['e1']
e3: ['e1', 'e2']
e4: ['e1', 'e2', 'e3']
e5: ['e1']
e6: ['e1', 'e2', 'e3', 'e4']
e7: ['e1', 'e5', 'e2', 'e3', 'e4', 'e6']
